# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import datetime

In [2]:
import api_token  as gwg # handle iexcloud API tokens

env = "cloud" # "sandbox"  # 
BASE_URL, IEX_CLOUD_API_TOKEN = gwg.get_base_url(env=env), gwg.get_api_token(env=env)

In [8]:
BATCH_SIZE = 100

WRITE_ALL_DF = True

TOP_N = 5
portfolio_size = 10000
file_csv = "stocks-dow30.csv"
filename=file_csv.replace(".csv", "")

In [9]:
today = datetime.datetime.now()
xlsx_file = f"spreadsheet_{filename}_top{TOP_N}_{today.strftime('%Y-%m-%d')}.xlsx"
xlsx_file2 = f"spreadsheet_{filename}_all_{today.strftime('%Y-%m-%d')}.xlsx"
watchlist_file = f"watchlist_{filename}_top{TOP_N}_{today.strftime('%Y-%m-%d')}.csv"

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [10]:

df = pd.read_csv(file_csv, 
                 names=["Ticker", "Company"], 
                 skiprows=[0])

In [11]:
stocks = df["Ticker"].values.tolist()

In [12]:
",".join(stocks)

'UNH,HD,GS,AMGN,CRM,BA,MSFT,MCD,V,HON,CAT,MMM,DIS,JNJ,WMT,PG,TRV,NKE,AAPL,IBM,JPM,AXP,CVX,MRK,VZ,DOW,KO,INTC,CSCO,WBA'

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [13]:
symbol_groups = list(gwg.chunks(stocks, BATCH_SIZE))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [14]:
hqm_columns = [
                'Ticker', 
                'Name',
                'Number of Shares to Buy', 
                'Price', 
                'Five-Day Price Return',
                'One-Month Price Return',
                'Three-Month Price Return',
                'Six-Month Price Return',
                'One-Year Price Return', 
                'Five-Day Return Percentile',
                'One-Month Return Percentile',
                'Three-Month Return Percentile',
                'Six-Month Return Percentile',
                'One-Year Return Percentile',
                'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                            pd.Series([symbol, 
                                       data[symbol]['stats']['companyName'],
                                       'N/A',
                                       data[symbol]['quote']['latestPrice'],
                                       data[symbol]['stats']['day5ChangePercent'],
                                       data[symbol]['stats']['month1ChangePercent'],
                                       data[symbol]['stats']['month3ChangePercent'],
                                       data[symbol]['stats']['month6ChangePercent'],
                                       data[symbol]['stats']['year1ChangePercent'],
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A'
                                       ], 
                                      index = hqm_columns), 
                            ignore_index = True)
        except Exception as ex:
            print(f"symbol = {symbol}\n Error={str(ex)}")
        
# hqm_dataframe.columns

Index(['Ticker', 'Name', 'Number of Shares to Buy', 'Price',
       'Five-Day Price Return', 'One-Month Price Return',
       'Three-Month Price Return', 'Six-Month Price Return',
       'One-Year Price Return', 'Five-Day Return Percentile',
       'One-Month Return Percentile', 'Three-Month Return Percentile',
       'Six-Month Return Percentile', 'One-Year Return Percentile',
       'HQM Score'],
      dtype='object')

In [15]:
# hqm_dataframe

In [16]:
time_periods = [
                'Five-Day',
                'One-Month',
                'Three-Month',
                'Six-Month',
                'One-Year'
                ]
price_return_cols = [f'{time_period} Price Return' for time_period in time_periods]

price_return_cols

['Five-Day Price Return',
 'One-Month Price Return',
 'Three-Month Price Return',
 'Six-Month Price Return',
 'One-Year Price Return']

In [17]:
hqm_dataframe = hqm_dataframe.dropna(how='any', 
                                     subset=price_return_cols)

In [18]:
# hqm_dataframe[hqm_dataframe["One-Year Price Return"].isnull()]

,Ticker,Name,Number of Shares to Buy,Price,Five-Day Price Return,One-Month Price Return,Three-Month Price Return,Six-Month Price Return,One-Year Price Return,Five-Day Return Percentile,One-Month Return Percentile,Three-Month Return Percentile,Six-Month Return Percentile,One-Year Return Percentile,HQM Score


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [19]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
# hqm_dataframe

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [20]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [21]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)

if WRITE_ALL_DF:
    hqm_dataframe_all = hqm_dataframe.copy()
    
hqm_dataframe = hqm_dataframe[:(TOP_N+1)]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [22]:
# portfolio_input()

In [23]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

In [24]:
position_size

1666.6666666666667

In [25]:
hqm_dataframe.reset_index(drop = True, inplace = True)

In [26]:
len(hqm_dataframe['Ticker'])

6

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [27]:
for i in range(len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [28]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')

In [29]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Name', string_template], 
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price', dollar_template],
                    'E': ['Five-Day Price Return', percent_template], 
                    'F': ['One-Month Price Return', percent_template], 
                    'G': ['Three-Month Price Return', percent_template],
                    'H': ['Six-Month Price Return', percent_template],
                    'I': ['One-Year Price Return', percent_template],
                    'J': ['Five-Day Return Percentile', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['Three-Month Return Percentile', percent_template],
                    'M': ['Six-Month Return Percentile', percent_template],
                    'N': ['One-Year Return Percentile', percent_template], 
                    'O': ['HQM Score', percent_template]
                    }

In [30]:
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [31]:
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [32]:
writer.save()

In [33]:
if WRITE_ALL_DF:
    writer = pd.ExcelWriter(xlsx_file2, engine='xlsxwriter')
    hqm_dataframe_all.to_excel(writer, sheet_name='Momentum Strategy', index = False)

    for column in column_formats.keys():
        writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
        writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

    writer.save()

### save watchlist

In [34]:
tickers = hqm_dataframe[['Ticker']]
tickers.to_csv(watchlist_file, index=False)